# Running RASA Chatbot in Colab

## RASA Main App

In [ ]:
!pip install rasa

In [ ]:
!git clone https://github.com/RasaHQ/rasa-for-beginners.git

Cloning into 'rasa-for-beginners'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 117 (delta 57), reused 117 (delta 57), pack-reused 0
Receiving objects: 100% (117/117), 376.04 KiB | 690.00 KiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
cd rasa-for-beginners

/content/rasa-for-beginners


In [ ]:
!rasa train

Training Core model...
2020-09-29 05:16:27 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
2020-09-29 05:16:27 INFO     root  - Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
Processed Story Blocks: 100% 10/10 [00:00<00:00, 3630.17it/s, # trackers=1]
Processed Story Blocks: 100% 10/10 [00:00<00:00, 515.19it/s, # trackers=10]
Processed Story Blocks: 100% 10/10 [00:00<00:00, 146.74it/s, # trackers=35]
Processed Story Blocks: 100% 10/10 [00:00<00:00, 150.66it/s, # trackers=30]
Processed trackers: 100% 10/10 [00:00<00:00, 4573.44it/s, # actions=34]
Processed actions: 34it [00:00, 12878.74it/s, # examples=34]
Processed trackers: 100% 510/510 [00:01<00:00, 422.11it/s, # actions=518]
2020-09-29 05:16:30.260075: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Epochs: 100% 100/100 [00:54<00:00,  1.82it/s, t_loss=0.086, loss=0.002, acc=1.000]


In [ ]:
# !pip install colab_ssh
from colab_ssh import launch_ssh
launch_ssh('<token>', '<pass>')

Successfully running 2.tcp.ngrok.io:19039
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

	Host google_colab_ssh
		HostName 2.tcp.ngrok.io
		User root
		Port 19039
	  


In [ ]:
!pip install -q pyngrok
from pyngrok import ngrok
!ngrok authtoken <token>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
ngrok.kill()
public_url = ngrok.connect(port='80'); public_url

'http://c41fc2bf36a0.ngrok.io'

In [ ]:
!rasa run -p 80

In [ ]:
# Twilio Endpoint
# http://c41fc2bf36a0.ngrok.io/webhooks/twilio/webhook

In [ ]:
!mv /content/*.py /content/rasa-for-beginners/files
!mv /content/*.ipynb /content/rasa-for-beginners/files

In [ ]:
%cd /content

/content


In [ ]:
# !mkdir -p /content/chatbots/wellnessTracker
# !mv  -v /content/rasa-for-beginners/* /content/chatbots/wellnessTracker/
# !rm -r /content/chatbots/wellnessTracker/.git

## RASA Action server

In [ ]:
# !pip install rasa
# !pip3 install python-dotenv

In [ ]:
!git clone https://github.com/RasaHQ/rasa-for-beginners.git

In [ ]:
%%writefile actions.py

from typing import Any, Text, Dict, List, Union
from dotenv import load_dotenv

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.forms import FormAction

import requests
import json
import os

load_dotenv()

airtable_api_key='<api-key>' #os.getenv("AIRTABLE_API_KEY")
base_id='<id>' #os.getenv("BASE_ID")
table_name='TABLEX' #os.getenv("TABLE_NAME")

def create_health_log(confirm_exercise, exercise, sleep, diet, stress, goal):
    request_url=f"https://api.airtable.com/v0/{base_id}/{table_name}"

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {airtable_api_key}",
    }  
    data = {
        "fields": {
            "Exercised?": confirm_exercise,
            "Type of exercise": exercise,
            "Amount of sleep": sleep,
            "Stress": stress,
            "Diet": diet,
            "Goal": goal,
        }
    }
    try:
        response = requests.post(
            request_url, headers=headers, data=json.dumps(data)
        )
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    
    return response
    print(response.status_code)

class HealthForm(FormAction):

    def name(self):
        return "health_form"

    @staticmethod
    def required_slots(tracker):

        if tracker.get_slot('confirm_exercise') == True:
            return ["confirm_exercise", "exercise", "sleep",
             "diet", "stress", "goal"]
        else:
            return ["confirm_exercise", "sleep",
             "diet", "stress", "goal"]

    def slot_mappings(self) -> Dict[Text, Union[Dict, List[Dict]]]:
        """A dictionary to map required slots to
            - an extracted entity
            - intent: value pairs
            - a whole message
            or a list of them, where a first match will be picked"""

        return {
            "confirm_exercise": [
                self.from_intent(intent="affirm", value=True),
                self.from_intent(intent="deny", value=False),
                self.from_intent(intent="inform", value=True),
            ],
            "sleep": [
                self.from_entity(entity="sleep"),
                self.from_intent(intent="deny", value="None"),
            ],
            "diet": [
                self.from_text(intent="inform"),
                self.from_text(intent="affirm"),
                self.from_text(intent="deny"),
            ],
            "goal": [
                self.from_text(intent="inform"),
            ],
        }

    def submit(
        self,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: Dict[Text, Any],
    ) -> List[Dict]:

        confirm_exercise = tracker.get_slot("confirm_exercise")
        exercise = tracker.get_slot("exercise")
        sleep = tracker.get_slot("sleep")
        stress = tracker.get_slot("stress")
        diet = tracker.get_slot("diet")
        goal = tracker.get_slot("goal")

        response = create_health_log(
                confirm_exercise=confirm_exercise,
                exercise=exercise,
                sleep=sleep,
                stress=stress,
                diet=diet,
                goal=goal
            )

        dispatcher.utter_message("Thanks, your answers have been recorded!")
        return []

Overwriting actions.py


In [ ]:
# !pip install -q pyngrok
# from pyngrok import ngrok
# !ngrok authtoken <token>
ngrok.kill()
public_url = ngrok.connect(port='5055'); public_url

'http://d40440ffde45.ngrok.io'

In [ ]:
!rasa run actions

2020-09-29 07:16:10 INFO     rasa_sdk.endpoint  - Starting action endpoint server...
2020-09-29 07:16:10 INFO     rasa_sdk.executor  - Registered function for 'health_form'.
2020-09-29 07:16:10 INFO     rasa_sdk.endpoint  - Action endpoint is up and running on http://localhost:5055
2020-09-29 09:13:26 ERROR    rasa_sdk.endpoint  - Failed to extract slot confirm_exercise with action health_form
2020-09-29 09:15:14 ERROR    rasa_sdk.endpoint  - Failed to extract slot confirm_exercise with action health_form
2020-09-29 09:32:13 ERROR    rasa_sdk.endpoint  - Failed to extract slot stress with action health_form
2020-09-29 09:34:28 ERROR    rasa_sdk.endpoint  - Failed to extract slot exercise with action health_form
